Explanation:
This code calculates the transmission load for each hour of every month, and writes it to the transmission_loads.csv text file. 

In [1]:
import numpy as np
import csv
import matplotlib.pyplot as plt
from scipy.stats import norm
import CoolProp as CP
from CoolProp.Plots import PropertyPlot
from CoolProp.CoolProp import PropsSI

#Average monthly temperatures per hour in degrees C. 
temp = {
    "January": [22.3, 21.3, 20.4, 19.8, 19.3, 19, 19.1, 21.2, 23.7, 26.2, 28.4, 30.2, 31.5, 32.4, 32.6, 32.2, 31.1, 29.5, 28.3, 27.3, 26.4, 25.3, 24.3, 23.3],
    "February": [25.8, 25, 24.1, 23.7, 23.2, 22.9, 23, 24.5, 26.4, 28.7, 30.9, 32.8, 34.4, 35.4, 35.7, 35.3, 34.1, 32.6, 31.2, 30.3, 29.4, 28.5, 27.6, 26.8],
    "March": [29.2, 28.5, 27.7, 27.3, 26.9, 26.7, 26.9, 28.5, 30.4, 32.4, 34.5, 36.4, 37.8, 38.6, 38.9, 38.4, 37.3, 35.9, 34.4, 33.5, 32.6, 31.8, 30.9, 30.1],
    "April": [29.8, 29, 28.6, 28.2, 27.9, 27.8, 28.4, 29.9, 31.6, 33.5, 35.2, 36.7, 37.8, 38.4, 38.7, 38.3, 37.4, 36.2, 34.9, 33.9, 33, 32.1, 31.2, 30.3],
    "May": [29.1, 28.4, 28.1, 27.8, 27.5, 27.5, 28.3, 29.7, 31.2, 32.8, 34.3, 35.5, 36.5, 37.0, 37.2, 37.0, 36.3, 35.1, 34.0, 33.1, 32.3, 31.4, 30.6, 29.8],
    "June": [26.6, 26.0, 25.7, 25.4, 25.1, 25.1, 25.9, 27.0, 28.4, 29.7, 31.0, 32.0, 32.8, 33.4, 33.6, 33.5, 33.0, 32.1, 31.0, 30.3, 29.5, 28.7, 27.9, 27.2],
    "July": [25.4, 24.8, 24.6, 24.2, 24.0, 23.9, 24.6, 25.6, 26.8, 28.0, 29.1, 30.0, 30.6, 31.1, 31.2, 31.1, 30.6, 30.0, 29.1, 28.4, 27.9, 27.2, 26.6, 25.9],
    "August": [24.5, 23.9, 23.7, 23.4, 23.3, 23.2, 23.8, 24.7, 26.0, 27.1, 28.3, 29.0, 29.6, 29.9, 30.0, 29.9, 29.4, 28.7, 27.9, 27.4, 26.7, 26.1, 25.7, 25.0],
    "September": [24.6, 23.9, 23.7, 23.4, 23.1, 23.1, 23.7, 25.0, 26.3, 27.6, 28.8, 29.8, 30.3, 30.7, 30.7, 30.5, 30.0, 29.2, 28.4, 27.7, 27.1, 26.4, 25.7, 25.0],
    "October": [26.6, 25.9, 25.5, 25.2, 24.9, 24.9, 25.6, 26.9, 28.6, 30.4, 31.9, 33.2, 34.1, 34.7, 34.8, 34.4, 33.5, 32.2, 31.4, 30.6, 29.7, 28.9, 28.1, 27.2],
    "November": [24.8, 24.0, 23.1, 22.6, 22.2, 21.9, 22.5, 25.2, 27.6, 29.9, 32.0, 33.6, 34.7, 35.3, 35.3, 34.5, 33.1, 31.3, 30.4, 29.4, 28.5, 27.5, 26.6, 25.6],
    "December": [23.3, 22.3, 21.4, 20.9, 20.4, 20.0, 20.3, 22.9, 25.4, 27.9, 30.0, 31.7, 33.0, 33.7, 33.8, 33.1, 31.8, 30.1, 29.1, 28.1, 27.1, 26.1, 25.1, 24.0],

}


# Constants
U = 0.23;  # Overall heat transfer coefficient, W/m^2*K
T_1 = 1;  # Inside temperature for room 1, degrees C
T_2 = 10;  # Inside temperature for room 2, Degrees C

# Solar heat gain in degrees C, if the cold storage is placed outside, change from 0. 
north = 0;
south = 0;
east = 0;
west = 0;
roof = 0;

#Temperature difference between the ambient and the floor
floor = -3;

#Areas for room 1: 
floor1 = 4.5;
south_wall1 = 0;
north_wall1= 7.5;
east_wall1 = 3.75;
west_wall1 = 3.75;
roof1 = 4.5;


# Areas for room 2:
floor2 = 4.5;
south_wall2 = 7.5;
north_wall2 = 0;
east_wall2 = 3.75;
west_wall2 = 3.75;
roof2 = 4.5;


transmission_loads = {}

# Loop through months and temperatures
for month, hourly_temps in temp.items():
    loads_per_hour = []
    
    for hour, temp_outside in enumerate(hourly_temps, start=1):
        delta_T1 = temp_outside - T_1
        delta_T2 = temp_outside - T_2

        if 8 <= hour <= 17: #Time with direct sunlight. Solar heat gain is set to 0, but can be changed if relevant. 
            trans_floor1 = U * floor1 * (delta_T1+floor)
            trans_roof1 = U * roof1 * (delta_T1)
            trans_south_wall1 = U * south_wall1 * (delta_T1)
            trans_north_wall1 = U * north_wall1 * (delta_T1)   
            trans_east_wall1 = U * east_wall1 * (delta_T1)           
            trans_west_wall1 = U * west_wall1* (delta_T1)                  

            trans_floor2 = U * floor2 * (delta_T2-floor)
            trans_roof2 = U * roof2 * (delta_T2+roof)
            trans_south_wall2 = U * south_wall2 * (delta_T2)
            trans_north_wall2 = U * north_wall2 * (delta_T2)   
            trans_east_wall2 = U * east_wall2 * (delta_T2)           
            trans_west_wall2 = U * west_wall2* (delta_T2) 
        else:
            trans_floor1 = U * floor1 * (delta_T1+floor)
            trans_roof1 = U * roof1 * (delta_T1)
            trans_south_wall1 = U * south_wall1 * (delta_T1)
            trans_north_wall1 = U * north_wall1 * (delta_T1)   
            trans_east_wall1 = U * east_wall1 * (delta_T1)           
            trans_west_wall1 = U * west_wall1* (delta_T1)                  

            trans_floor2 = U * floor2 * (delta_T2)
            trans_roof2 = U * roof2 * (delta_T2)
            trans_south_wall2 = U * south_wall2 * (delta_T2)
            trans_north_wall2 = U * north_wall2 * (delta_T2)   
            trans_east_wall2 = U * east_wall2 * (delta_T2)           
            trans_west_wall2 = U * west_wall2* (delta_T2) 

        
        total_load_room1 = trans_floor1 + trans_roof1 + trans_south_wall1 + trans_north_wall1 + trans_east_wall1 + trans_west_wall1;
        total_load_room2 = trans_floor2 + trans_roof2 + trans_south_wall2 + trans_north_wall2 + trans_east_wall2 + trans_west_wall2;
        
        loads_per_hour.append((hour, total_load_room1, total_load_room2))

    
    transmission_loads[month] = loads_per_hour



for month, loads in transmission_loads.items():
    print(f"{month} Transmission Loads:")
    for hour, load1, load2 in loads:
        print(f"  {hour:02}:00 , total load: {load1+load2:.2f} W")


#Output file name
output_file = "transmission_loads.csv"
with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    

    writer.writerow(["Month", "Hour", "Total Load Room 1 (W)", "Total Load Room 2 (W)", "Total Load (W)"])
    for month, loads in transmission_loads.items():
        for hour, load1, load2 in loads:
            writer.writerow([month, hour, f"{load1/1000:.2f}", f"{load2/1000:.2f}", f"{(load1 + load2)/1000:.2f}"])

print(f"Transmission loads saved to {output_file}")


January Transmission Loads:
  01:00 , total load: 182.37 W
  02:00 , total load: 171.33 W
  03:00 , total load: 161.39 W
  04:00 , total load: 154.77 W
  05:00 , total load: 149.25 W
  06:00 , total load: 145.94 W
  07:00 , total load: 147.04 W
  08:00 , total load: 173.33 W
  09:00 , total load: 200.93 W
  10:00 , total load: 228.53 W
  11:00 , total load: 252.82 W
  12:00 , total load: 272.69 W
  13:00 , total load: 287.04 W
  14:00 , total load: 296.98 W
  15:00 , total load: 299.18 W
  16:00 , total load: 294.77 W
  17:00 , total load: 282.62 W
  18:00 , total load: 261.86 W
  19:00 , total load: 248.61 W
  20:00 , total load: 237.57 W
  21:00 , total load: 227.63 W
  22:00 , total load: 215.49 W
  23:00 , total load: 204.45 W
  24:00 , total load: 193.41 W
February Transmission Loads:
  01:00 , total load: 221.01 W
  02:00 , total load: 212.18 W
  03:00 , total load: 202.24 W
  04:00 , total load: 197.82 W
  05:00 , total load: 192.30 W
  06:00 , total load: 188.99 W
  07:00 , tot